In [2]:
import pandas as pd
import json

In [2]:
with open('../data/variables/2002/ZA3880_variables.json', 'r') as f:
    data_02 = json.load(f)

In [10]:
with open('../data/var_02_map.json', 'r') as f:
    var_02 = json.load(f)

var_02 = {key.upper() : value for key, value in var_02.items()}

In [11]:
questions_02 = {}

for var_name, data_info in data_02['variables'].items():
    if var_name.upper() in var_02:
        questions_02[var_name] = {'question': data_info['question_en'], 'details':data_info['notes_en']}

In [12]:
len(questions_02)

45

In [13]:
import pprint


pprint.pprint(questions_02)

{'COUNTRY': {'details': 'Country/ Sample (ISO 3166)<br/><br/>This variable '
                        'holds the country codes for the participating '
                        'countries. Since these country codes are a mere '
                        'enumneration, the codes/ values themselves do not '
                        'contain ISO 3166 information. Although the country '
                        'abreviations used in the value labels are closely '
                        'adjusted to ISO 3166. For more information regarding '
                        'this ISO standard, please see variable '
                        'C_ALPHAN.<br/><br/>The country codes are incorporated '
                        'in variable V3, Respondent Number.<br/>',
             'question': 'Country/ Sample (ISO 3166)'},
 'C_ALPHAN': {'details': 'ISO 3166 Country/ Sample Prefix<br/><br/>This '
                         'alphanumerical sample identification variable '
                         'C_ALPHAN includes c

In [69]:


# SIMILARITY_THRESHOLD = 0.85
# MODEL_NAME = "all-MiniLM-L6-v2"

# FILES = {
#     "02": "../data_dump/variables/2002/ZA3880_variables_short.json",
#     "12": "../data_dump/variables/2012/ZA5900_variables_short.json",
#     "22": "../data_dump/variables/2022/ZA10000_variables_short.json",
# }

# def load_unique_questions(path):
#     with open(path, "r", encoding="utf-8") as f:
#         data = json.load(f)

#     # Exact-text deduplication only
#     q_to_var = {}
#     for var, q in data.items():
#         q = q.strip()
#         if q not in q_to_var:
#             q_to_var[q] = var
#     return q_to_var


# # -------------------------------
# # LOAD DATA
# # -------------------------------
# records = []
# for year, path in FILES.items():
#     qmap = load_unique_questions(path)
#     for q, var in qmap.items():
#         records.append({"year": year, "question": q, "var": var})

# meta_df = pd.DataFrame(records)

# # -------------------------------
# # EMBEDDINGS
# # -------------------------------
# model = SentenceTransformer(MODEL_NAME)
# embeddings = model.encode(meta_df["question"].tolist(), normalize_embeddings=True)

# # -------------------------------
# # DISTANCE MATRIX WITH YEAR CONSTRAINT
# # -------------------------------
# cos_sim = cosine_similarity(embeddings)
# distance = 1 - cos_sim

# years = meta_df["year"].values

# # 🚫 Prevent same-year merging unless exact text match
# for i in range(len(distance)):
#     for j in range(len(distance)):
#         if i != j and years[i] == years[j]:
#             if meta_df.iloc[i]["question"] != meta_df.iloc[j]["question"]:
#                 distance[i, j] = 1.0  # max distance → cannot merge

# # -------------------------------
# # CLUSTERING
# # -------------------------------
# clustering = AgglomerativeClustering(
#     metric="precomputed",
#     linkage="average",
#     distance_threshold=1 - SIMILARITY_THRESHOLD,
#     n_clusters=None,
# )

# labels = clustering.fit_predict(distance)
# meta_df["group"] = labels

# # -------------------------------
# # BUILD FINAL TABLE
# # -------------------------------
# rows = []

# for gid, group in meta_df.groupby("group"):
#     row = {
#         "question": group.iloc[0]["question"],
#         "question_02": "NA",
#         "question_12": "NA",
#         "question_22": "NA",
#         "var_02": "NA",
#         "var_12": "NA",
#         "var_22": "NA",
#         "similarity_score": round(
#             cosine_similarity(
#                 model.encode(group["question"].tolist(), normalize_embeddings=True)
#             ).mean(),
#             3,
#         ),
#     }

#     for _, r in group.iterrows():
#         row[f"question_{r['year']}"] = r["question"]
#         row[f"var_{r['year']}"] = r["var"]

#     rows.append(row)

# final_df = pd.DataFrame(rows)
# final_df.to_csv("common_question_mapping.csv", index=False)


In [5]:
# Load the survey data
df_02 = pd.read_stata('../data/dta_files/2002.dta', convert_categoricals=False)

# # Load variable descriptions
# with open('../data_dump/var_02_map.json', 'r') as f:
#     variables_02 = json.load(f)

# variables_02.pop('NaN')
# df_02.rename(columns={col: col.upper() for col in  df_02.columns}, inplace=True)
# df_02.drop(columns=[i for i in df_02.columns if i not in variables_02.keys()], inplace=True, errors='ignore')
# df_02.rename(columns={old: new for old, new in variables_02.items() if old in df_02.columns}, inplace=True)
# df_02.head()

In [11]:
df_02.columns

Index(['v1', 'v2', 'v3', 'COUNTRY', 'C_ALPHAN', 'v4', 'v5', 'v6', 'v7', 'v8',
       ...
       'v352', 'v353', 'v354', 'v355', 'v356', 'v357', 'v358', 'v359', 'v360',
       'v361'],
      dtype='object', length=235)

In [21]:
df_02[df_02['C_ALPHAN']=='CL']['v48'].describe()

count    774.000000
mean       2.114987
std        0.903326
min        1.000000
25%        1.000000
50%        2.000000
75%        3.000000
max        4.000000
Name: v48, dtype: float64

In [30]:
with pd.io.stata.StataReader("../data_dump/dta_files/2002.dta", convert_categoricals=True) as rdr:
    cols = [c for c in rdr.variable_labels().keys() if c not in ["v241","v247"]]
    df_2002 = rdr.read(columns=cols)

# Load variable descriptions
with open('../data_dump/var_02_map.json', 'r') as f:
    variables_02 = json.load(f)

variables_02 = {key.upper(): values for key,values  in variables_02.items()}
df_2002.rename(columns={col: col.upper() for col in  df_2002.columns}, inplace=True)

if 'NaN' in variables_02:
    variables_02.pop('NaN')


df_2002.drop(columns=[i for i in df_2002.columns if i not in variables_02.keys()], inplace=True)
df_2002.rename(columns={old: new for old, new in variables_02.items() if old in df_2002.columns}, inplace=True)
df_2002.head()

,CASEID,COUNTRY,C_ALPHAN,WW_WARM,WW_CHILD_SUFFER,WW_FAM_SUFFER,WO_WANT,HW_FULFIL,MEWH,MOMORFAF,...,age,marital,COHAB,highest_education,work_status,wrk_hrs,spouse_work_status,HOMPOP,TOPBOT,urban_rural
0,1000001,Australia (AU),AU,Disagree,Agree,Agree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,Neither agree nor disagree,...,46.0,"Marr,liv as mar",NaN,Above lowest qualification,Employed-part time,30.0,Retired,2 persons,5.0,"Urban,a big city"
1,1000002,Australia (AU),AU,Strongly agree,Strongly disagree,Strongly disagree,Strongly disagree,Strongly agree,Strongly disagree,Strongly agree,...,40.0,"Marr,liv as mar",NaN,University degree completed,Unemployed,NaN,"F-t empl,main job",2 persons,8.0,"Urban,a big city"
2,1000003,Australia (AU),AU,Agree,Neither agree nor disagree,Agree,Neither agree nor disagree,Agree,Disagree,Agree,...,68.0,"Marr,liv as mar",NaN,Above lowest qualification,Retired,NaN,Retired,2 persons,6.0,"Urban,a big city"
3,1000004,Australia (AU),AU,Disagree,Agree,Strongly agree,Disagree,Disagree,Disagree,Strongly agree,...,69.0,"Marr,liv as mar",NaN,University degree completed,Retired,NaN,Retired,2 persons,4.0,"Urban,a big city"
4,1000005,Australia (AU),AU,Neither agree nor disagree,Agree,Agree,Neither agree nor disagree,Agree,Neither agree nor disagree,Agree,...,44.0,"Marr,liv as mar",NaN,"Above higher sec level,below full uni",Employed-part time,8.0,"F-t empl,main job",2 persons,7.0,"Urban,a big city"


In [35]:
df_2022 = pd.read_stata("../data_dump/dta_files/2022.dta")

# Load variable descriptions
with open('../data_dump/var_22_map.json', 'r') as f:
    variables_22 = json.load(f)
    
variables_22 = {key.upper(): values for key,values  in variables_22.items()}
df_2022.rename(columns={col: col.upper() for col in  df_2022.columns}, inplace=True)

if 'NaN' in variables_22:
    variables_22.pop('NaN')


df_2022.drop(columns=[i for i in df_2022.columns if i not in variables_22.keys()], inplace=True)
df_2022.rename(columns={old.upper(): new for old, new in variables_22.items() if old.upper() in df_2022.columns}, inplace=True)
df_2022.head()

,COUNTRY,C_ALPHAN,WW_WARM,WW_CHILD_SUFFER,WW_FAM_SUFFER,WO_WANT,HW_FULFIL,MEWH,MOMORFAF,WWYKUS,...,SPWRKHRS,spouse_work_status,TOPBOT,HOMPOP,HHADULT,HHCHILDR,HHTODD,marital,urban_rural,CASEID
0,40. AT-Austria,AT,1. Strongly agree,5. Strongly disagree,5. Strongly disagree,5. Strongly disagree,3. Neither agree nor disagree,5. Strongly disagree,3. Mothers and fathers are equally suited,1. Work full-time,...,"-4. NAP (code -4, 2 or 3 in SPWORK)","-4. NAP (c.3,-7 PARTLIV; US: c. 2 (if neither ...",4. 04,2. Two persons,1. One adult (only respondent),1. One child [school entry age] to 17,0. No children below [age of school entry],4. Divorced from spouse/legally separated from...,2. The suburbs or outskirts of a big city,2.022000e+15
1,40. AT-Austria,AT,4. Disagree,1. Strongly agree,2. Agree,2. Agree,1. Strongly agree,2. Agree,2. Mothers are somewhat better suited,2. Work part-time,...,"-4. NAP (code -4, 2 or 3 in SPWORK)","-4. NAP (c.3,-7 PARTLIV; US: c. 2 (if neither ...","1. Lowest, Bottom, 01",1. One person (only respondent),1. One adult (only respondent),0. No children [school entry age] to 17,0. No children below [age of school entry],4. Divorced from spouse/legally separated from...,3. A town or a small city,2.022000e+15
2,40. AT-Austria,AT,4. Disagree,1. Strongly agree,1. Strongly agree,2. Agree,3. Neither agree nor disagree,3. Neither agree nor disagree,3. Mothers and fathers are equally suited,3. Stay at home,...,"-4. NAP (code -4, 2 or 3 in SPWORK)","-4. NAP (c.3,-7 PARTLIV; US: c. 2 (if neither ...",8. 08,4. Four persons,4. Four adults,0. No children [school entry age] to 17,0. No children below [age of school entry],6. Never married/never in a civil partnership,4. A country village,2.022000e+15
3,40. AT-Austria,AT,5. Strongly disagree,2. Agree,2. Agree,1. Strongly agree,1. Strongly agree,1. Strongly agree,2. Mothers are somewhat better suited,3. Stay at home,...,40,1. In paid work,5. 05,4. Four persons,4. Four adults,0. No children [school entry age] to 17,0. No children below [age of school entry],1. Married,1. A big city,2.022000e+15
4,40. AT-Austria,AT,1. Strongly agree,5. Strongly disagree,4. Disagree,5. Strongly disagree,5. Strongly disagree,5. Strongly disagree,3. Mothers and fathers are equally suited,1. Work full-time,...,45,1. In paid work,7. 07,4. Four persons,2. Two adults,2. Two children [school entry age] to 17,0. No children below [age of school entry],1. Married,1. A big city,2.022000e+15
